In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

In [ ]:
# create a function to clean the acquired text
def cleaning_text(text: str):
    # remove whitespace
    text_wo_whitespace = re.sub(r'\s+', ' ', text)
    # add space after period
    proper_sentences = re.sub(r'\.([a-zA-Z]+)\b', r'. \1', text_wo_whitespace)
    # split paragraph into sentences
    sentences = proper_sentences.split('. ')
    # filter irrelevant sentences
    filtered_sentences = filter(
        lambda sentence: "ADVERTISEMENT" not in sentence and "Baca juga:" not in sentence and "Simak Video" not in sentence, sentences
    )
    # join the sentences back into a full text
    cleaned_text = '. '.join(filtered_sentences)

    return cleaned_text

In [ ]:
URL = "https://inet.detik.com/indeks"

full_data = []

for page in range(1, 251):
    resp = requests.get(f'{URL}/{page}')
    if resp.status_code == 200:
        soup = BeautifulSoup(resp.text, 'html.parser')

        # get all the articles
        medias = soup.find_all('div', class_='media')

        for media in medias:
            # get the article's link
            link = media.find('a', class_='media__link')
            href = link.get('href')

            # filter out videos and pictures articles
            if "20.detik.com" not in href and 'fotoinet' not in href:
                content = requests.get(href)
                content_soup = BeautifulSoup(content.text, 'html.parser')
                content_detail = content_soup.find('article', class_='detail')

                if content_detail:
                    content_title = content_detail.find('h1', class_='detail__title').text.strip() or None
                    content_author = content_detail.find('div', class_='detail__author').text.replace(' - detikInet', '') or None
                    content_date = content_detail.find('div', class_= 'detail__date').text or None
                    
                    content_body = content_detail.find('div', class_='detail__body').text.strip() or None
                    cleaned_content_body = None
                    if content_body:
                        cleaned_content_body = cleaning_text(content_body)

                    content_data = {
                        'title': content_title,
                        'author': content_author,
                        'date': content_date,
                        'body': cleaned_content_body
                    }

                    full_data.append(content_data)

In [ ]:
# turn the acquired data into a Data Frame object
full_data_df = pd.DataFrame(full_data)

In [ ]:
# export Data Frame into a CSV file
full_data_df.to_csv('data/detik_inet.csv', index=False)